In [1]:
from pykrx import stock
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

In [7]:
# Data Prerprocessing
samsung_code = "005930"    
skhynix_code = "000660"  

start_date = "2017-10-19"
end_date = "2019-10-31"

samsung_df = stock.get_market_ohlcv_by_date(start_date, end_date, samsung_code)
skhynix_df = stock.get_market_ohlcv_by_date(start_date, end_date, skhynix_code)


df = pd.DataFrame({'samsumg': samsung_df['종가'], 'skhynix': skhynix_df['종가']}).astype(float)



In [8]:
def monte_carlo_simulation(drift, volatility, epsilon, num_assets):
    dt = 1/252  # 일일 시간 간격
    log_returns = []
    for i in range(num_assets):
        log_return = (drift - 0.5 * volatility[i]**2) * dt + volatility[i] * np.sqrt(dt) * epsilon[i]
        log_returns.append(log_return)
    return np.array(log_returns)

In [9]:
def find_f1(worst_performer, index, rf, first, second, third, due_first, due_second, due_third, due_last, EC_strike_rate, first_payoff, second_payoff, third_payoff, f, initial_price):
    if index < first and worst_performer[due_first] > EC_strike_rate:
            f.append(first_payoff*initial_price[0] / np.exp(rf/252 * due_first))
    elif index < second and worst_performer[due_second] > EC_strike_rate:
        f.append(second_payoff*initial_price[0] / np.exp(rf/252 * due_second))
    elif index < third and worst_performer[due_third] > EC_strike_rate:
        f.append(third_payoff*initial_price[0] / np.exp(rf/252 * due_third))                  
    else:
        max_return = max(worst_performer[due_last], 0.95)
        f.append(max_return*initial_price[0] / np.exp(rf/252 * due_last))
    return

def find_f2(worst_performer, index, rf, first, second, third, due_first, due_second, due_third, due_last, EC_strike_rate, first_payoff, second_payoff, third_payoff, f, initial_price):
    if index < first and worst_performer[due_first] > EC_strike_rate:
            f.append(first_payoff*initial_price[1] / np.exp(rf/252 * due_first))
    elif index < second and worst_performer[due_second] > EC_strike_rate:
        f.append(second_payoff*initial_price[1] / np.exp(rf/252 * due_second))
    elif index < third and worst_performer[due_third] > EC_strike_rate:
        f.append(third_payoff*initial_price[1] / np.exp(rf/252 * due_third))                  
    else:
        max_return = max(worst_performer[due_last], 0.95)
        f.append(max_return*initial_price[1] / np.exp(rf/252 * due_last))
    return

In [10]:
num_assets = 2
time_steps = 252
num_samples = 10000
rf = 0.02

df_index = df.iloc[252::20, :]
initial_price = df.iloc[252]
EC_strike_rate = 1.02
MC_strike_rate = 1.00
EC_strike_price = initial_price * EC_strike_rate
MC_strike_price = initial_price * MC_strike_rate
dt = 20/252

ELS_Delta1 = []
ELS_Delta2 = []

### 2018-10-31부터 20일단위로 rebalancing 진행
for index, names in df_index.iterrows():    
    print(index)
    simulated_S = []
    simulated_Su = []
    simulated_Sd = []
    
    # 리밸런싱 시점의 252 영업일 전부터의 데이터 가져오기
    init_price = df.loc[index]
    index_num = df.index.get_loc(index)
    df_hist = df.iloc[index_num-252:index_num, :]
        
    # MCMC로 10000개의 (2, 252) 누적 수익률 데이터 만들기
    cholesky_mat = np.linalg.cholesky(df_hist.corr())
    vol = np.log(df_hist/df_hist.shift(1))[1:].std() * np.sqrt(252)
        
    for _ in range(num_samples):                        
        epsilon = np.dot(cholesky_mat, np.random.randn(num_assets, time_steps))
        simulated_returns = monte_carlo_simulation(rf, vol, epsilon, num_assets)
        
        S = np.exp(np.cumsum((simulated_returns), axis=1))
        #S[0] *= init_price[0]
        #S[1] *= init_price[1]        
        Su = S.copy()
        Su[0] *= np.exp(vol[0]*np.sqrt(dt))
        Su[1] *= np.exp(vol[1]*np.sqrt(dt))
        Sd = S.copy()
        Sd[0] *= np.exp(-1*vol[0]*np.sqrt(dt))
        Sd[1] *= np.exp(-1*vol[1]*np.sqrt(dt))
        
        simulated_S.append(S)
        simulated_Su.append(Su)
        simulated_Sd.append(Sd)
                
    simulated_S = np.array(simulated_S)
    simulated_Su = np.array(simulated_Su)
    simulated_Sd = np.array(simulated_Sd)       
    
        
    ### ELS 가치평가
    first = datetime.strptime('2019-01-31', '%Y-%m-%d')
    second = datetime.strptime('2019-04-30', '%Y-%m-%d')
    third = datetime.strptime('2019-07-31', '%Y-%m-%d')
    last = datetime.strptime('2019-10-31', '%Y-%m-%d')
    
    f1_up = []
    f1_down = []
    f2_up = []
    f2_down = []
        
    first_payoff = 1.0159
    second_payoff = 1.0318
    third_payoff = 1.0477
        
    due_first = len(df.loc[:first]) - len(df.loc[:index])
    due_second = len(df.loc[:second]) - len(df.loc[:index])
    due_third = len(df.loc[:third]) - len(df.loc[:index])
    due_last = len(df.loc[:last]) - len(df.loc[:index])
        
    for S, Su, Sd in zip(simulated_S, simulated_Su, simulated_Sd):        
        
        # f1_up 구하기 : 기초자산1 상승, 기초자산2 불변        
        worst_performer = np.min(np.vstack((Su[0], S[1])), axis=0)        
        find_f1(worst_performer, index, rf, first, second, third, due_first, due_second, due_third, due_last, EC_strike_rate, first_payoff, second_payoff, third_payoff, f1_up, initial_price)        
        
        # f1_down 구하기 : 기초자산1 하락, 기초자산2 불변
        worst_performer = np.min(np.vstack((Sd[0], S[1])), axis=0)
        find_f1(worst_performer, index, rf, first, second, third, due_first, due_second, due_third, due_last, EC_strike_rate, first_payoff, second_payoff, third_payoff, f1_down, initial_price)        
                
        # f2_up 구하기 : 기초자산1 불변, 기초자산2 상승
        worst_performer = np.min(np.vstack((S[0], Su[1])), axis=0)
        find_f2(worst_performer, index, rf, first, second, third, due_first, due_second, due_third, due_last, EC_strike_rate, first_payoff, second_payoff, third_payoff, f2_up, initial_price)        
        
        # f2_down 구하기 : 기초자산1 불변, 기초자산2 하락
        worst_performer = np.min(np.vstack((S[0], Sd[1])), axis=0)
        find_f2(worst_performer, index, rf, first, second, third, due_first, due_second, due_third, due_last, EC_strike_rate, first_payoff, second_payoff, third_payoff, f2_down, initial_price)        
        
    # Els Delta 구하기    
    f1_up = sum(f1_up)/num_samples
    f1_down = sum(f1_down)/num_samples
    f2_up = sum(f2_up)/num_samples
    f2_down = sum(f2_down)/num_samples
    
    print(f1_up)
    print(f1_down)    
    delta_1 = (f1_up - f1_down) / (init_price[0] * np.exp(vol[0]*np.sqrt(dt)) - init_price[0] * np.exp(-1*vol[0]*np.sqrt(dt)))
    delta_2 = (f2_up - f2_down) / (init_price[1] * np.exp(vol[1]*np.sqrt(dt)) - init_price[1] * np.exp(-1*vol[1]*np.sqrt(dt)))
    ELS_Delta1.append(delta_1)
    ELS_Delta2.append(delta_2)

2018-10-31 00:00:00
41629.56478725314
41011.39096700948
2018-11-28 00:00:00
41859.84596061303
41170.4070340559
2018-12-27 00:00:00
42158.31199515985
41449.24873584482
2019-01-28 00:00:00
42426.32759698393
41583.169665677204
2019-02-28 00:00:00
42462.8321168324
41731.608360326565
2019-03-29 00:00:00
42687.75798124999
41810.96523258022
2019-04-26 00:00:00
42868.59208438772
41723.22815139442
2019-05-28 00:00:00
42885.76324175066
41861.668319204015
2019-06-26 00:00:00
43003.74695618795
41758.113511169264
2019-07-24 00:00:00
43381.15439475275
41576.451922345026
2019-08-22 00:00:00
43386.47937497234
41404.129075739584
2019-09-23 00:00:00
43073.63794598348
41074.581236075006
2019-10-23 00:00:00
42637.718498250986
40640.830637764106


In [11]:
print("Asset 1\n", ELS_Delta1)
print("Asset 2\n", ELS_Delta2)

Asset 1
 [0.09407903000330896, 0.1031862349154668, 0.12080646961501933, 0.12187794460687482, 0.10856260628146834, 0.1331044815170041, 0.1779514855037434, 0.16963619790420567, 0.19694348420546307, 0.28365537368745347, 0.3261746314487672, 0.29459010740402525, 0.2937905606551958]
Asset 2
 [0.07156520562808058, 0.07324655631985062, 0.09182545971220239, 0.08324636282955185, 0.07235669453358255, 0.08297041563777113, 0.09331126656570678, 0.09883083980318493, 0.11818185345194725, 0.14507189987471078, 0.17188057137690949, 0.15302999580460863, 0.19705742859853995]


In [66]:
"""
cumulative_log_returns = np.cumsum(simulated_returns, axis=1)
print(cumulative_log_returns.shape)

# 결과 시각화
plt.figure(figsize=(16, 9))
plt.plot(cumulative_log_returns[0, :], label='Samsumg')
plt.plot(cumulative_log_returns[1, :], label='Skhynix')
plt.title('Monte Carlo Simulation of Cumulative Asset Returns')
plt.xlabel('Days')
plt.ylabel('Cumulative Asset Returns')
plt.show() 
"""


"\ncumulative_log_returns = np.cumsum(simulated_returns, axis=1)\nprint(cumulative_log_returns.shape)\n\n# 결과 시각화\nplt.figure(figsize=(16, 9))\nplt.plot(cumulative_log_returns[0, :], label='Samsumg')\nplt.plot(cumulative_log_returns[1, :], label='Skhynix')\nplt.title('Monte Carlo Simulation of Cumulative Asset Returns')\nplt.xlabel('Days')\nplt.ylabel('Cumulative Asset Returns')\nplt.show() \n"